# Configure Zero Latency Raw Bridge From Windows Camera to Linux


##### Download and Install FFMPEG

```powershell
New-Item -ItemType Directory -Force -Path "C:\Tools"
Set-Location "C:\Tools"
Invoke-WebRequest -Uri "https://www.gyan.dev/ffmpeg/builds/ffmpeg-release-essentials.zip" -OutFile "ffmpeg.zip"
Expand-Archive -Path "ffmpeg.zip" -DestinationPath "C:\Tools\ffmpeg" -Force
Set-Location "C:\Tools"
Invoke-WebRequest -Uri "https://www.gyan.dev/ffmpeg/builds/ffmpeg-release-essentials.zip" -OutFile "ffmpeg.zip"
Expand-Archive -Path "ffmpeg.zip" -DestinationPath "C:\Tools\ffmpeg" -Force
```

#### Find the path

```powershell
PS C:\Tools> Get-ChildItem "C:\Tools\ffmpeg" -Recurse -Filter ffmpeg.exe | Select-Object -First 1 FullName
```

#### output FullName
    C:\Tools\ffmpeg\ffmpeg-8.0-essentials_build\bin\ffmpeg.exe
    
#### Test Path

```powershell
PS C:\Tools> & "C:\Tools\ffmpeg\ffmpeg-8.0-essentials_build\bin\ffmpeg.exe" -version
```

#### Set Environment Variable and List Devices

```powershell
PS C:\Tools> $env:FFMPEG="C:\Tools\ffmpeg\ffmpeg-8.0-essentials_build\bin\ffmpeg.exe" # Set FFMPEG environment variable
PS C:\Tools> & $env:FFMPEG -list_devices true -f dshow -i dummy # List devicesa
```

#### List Supported Formats for Camera

```powershell
& $env:FFMPEG -f dshow -list_options true -i video="USB Camera Name" # see supported formats
```


# Install Python packages

ultralytics, onnxruntime, onnx, opencv-python, numpy, Pillow, tqdm, simple-lama-inpainting, sympy, matplotlib, seaborn, pandas, ipywidgets .venv/bin/python -c "from ultralytics import YOLO; YOLO('yolov8n-seg.pt')"

## Receive the Stream in WSL

**Correct sequence:**

1. Run the WSL server cell **first** —it will print "Waiting for Windows connection (timeout in 30 seconds)..."
2. Within 30 seconds, run the ffmpeg command on Windows:

```
& "C:\Tools\ffmpeg\ffmpeg-8.0-essentials_build\bin\ffmpeg.exe" ^
  -f dshow ^
  -rtbufsize 256M ^
  -video_size 1280x720 ^
  -framerate 30 ^
  -i video="USB Camera" ^
  -pix_fmt bgr24 ^
  -fflags nobuffer ^
  -flags low_delay ^
  -loglevel warning ^
  -f rawvideo ^
  "tcp://172.24.146.232:5000"
```
# Behavior:
 -  Starts listening on port 5000
 -  Waits 30 seconds for a connection from Windows ffmpeg
 -  During this wait, the cell is "blocked" - it doesn't proceed until either: 
 -- Windows connects (success)
 -- 30 seconds timeout (failure)

What happens during wait: Nothing - it's just listening for incoming TCP connection

In [5]:
# listen_raw_server.py with integrated processing
import socket, numpy as np, cv2
from ultralytics import YOLO
from simple_lama_inpainting import SimpleLama
import time
import select

HOST, PORT = "0.0.0.0", 5000

W,H = 1280,720 # Set resolution here Must Match Client
BYTES = W*H*3

# Initialize frame counter
frame_count = 0
max_frames = 30  # Process 30 frames

# Options
save_videos = False  # Set to True to save videos, False for live-only

# Load YOLO model
yolo_model = YOLO('yolov8n-seg.pt')

# LaMa model
lama_model = SimpleLama()

# Initialize video writers if saving
if save_videos:
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    out_original = cv2.VideoWriter('./Original_Webcam.avi', fourcc, 30, (W, H))
    out_inpainted = cv2.VideoWriter('./SegYolov8_InptSimLama.avi', fourcc, 30, (W, H))

# Create server socket
srv = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
srv.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
srv.bind((HOST, PORT))
srv.listen(1)

print("Waiting for Windows connection (timeout in 30 seconds)...")

ready = select.select([srv], [], [], 30)

if ready[0]:
    conn, addr = srv.accept()
    print(f"Connected to {addr}")
    
    buf = bytearray(BYTES)
    mv = memoryview(buf)
    start_time = time.time()
    
    while frame_count < max_frames:
        got = 0
        while got < BYTES:
            n = conn.recv_into(mv[got:], BYTES - got)
            if n == 0:
                print("Connection closed by client")
                break
            got += n
        if got < BYTES:
            break
       # Convert buffer to image 
        frame = np.frombuffer(buf, np.uint8).reshape((H, W, 3)) # BGR format
        
        if save_videos:
            # Write original frame
            out_original.write(frame)
        
        # YOLO inference for people (class 0)
        results = yolo_model(frame, classes=[0], verbose=False)
        
        print(f"Frame {frame_count}: YOLO detected {len(results[0].masks) if results[0].masks is not None else 0} people")
        
        # Create mask
        if results[0].masks is not None and len(results[0].masks) > 0:
            mask = results[0].masks.data[0].cpu().numpy()
            mask = (mask * 255).astype(np.uint8)
            mask = cv2.resize(mask, (W, H), interpolation=cv2.INTER_NEAREST)
        else:
            # No people, skip or use empty mask
            mask = np.zeros((H, W), dtype=np.uint8)
        
        # Inpaint using LaMa
        inpainted_result = lama_model(frame, mask)
        if isinstance(inpainted_result, np.ndarray):
            inpainted_frame = inpainted_result
        else:
            inpainted_frame = np.array(inpainted_result)
            if len(inpainted_frame.shape) == 3 and inpainted_frame.shape[2] == 3:
                inpainted_frame = cv2.cvtColor(inpainted_frame, cv2.COLOR_RGB2BGR)
        
        if save_videos:
            # Write inpainted frame
            out_inpainted.write(inpainted_frame)
        
        # Display real-time side-by-side feed
        combined_frame = np.concatenate((frame, inpainted_frame), axis=1)
        cv2.imshow('Real-time Side-by-Side Feed', combined_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("User pressed 'q' to quit real-time display")
            break
        
        frame_count += 1
        elapsed = time.time() - start_time
        fps = frame_count / elapsed if elapsed > 0 else 0
        if frame_count % 10 == 0:
            print(f"Processed {frame_count} frames... FPS: {fps:.2f}")
    
    conn.close()
else:
    print("Timeout: No connection from Windows")

srv.close()
if save_videos:
    out_original.release()
    out_inpainted.release()
cv2.destroyAllWindows()

print(f"Processing complete! Processed {frame_count} frames.")
if save_videos:
    print("Videos saved: ./Original_Webcam.avi and ./SegYolov8_InptSimLama.avi")
else:
    print("Videos not saved (save_videos=False)")

Waiting for Windows connection (timeout in 30 seconds)...
Connected to ('172.24.144.1', 49548)
Frame 0: YOLO detected 1 people
Frame 0: YOLO detected 1 people
Frame 1: YOLO detected 1 people
Frame 1: YOLO detected 1 people
Frame 2: YOLO detected 1 people
Frame 3: YOLO detected 1 people
Frame 2: YOLO detected 1 people
Frame 3: YOLO detected 1 people
Frame 4: YOLO detected 1 people
Frame 5: YOLO detected 1 people
Frame 4: YOLO detected 1 people
Frame 5: YOLO detected 1 people
Frame 6: YOLO detected 1 people
Frame 7: YOLO detected 1 people
Frame 6: YOLO detected 1 people
Frame 7: YOLO detected 1 people
Frame 8: YOLO detected 1 people
Frame 9: YOLO detected 1 people
Frame 8: YOLO detected 1 people
Frame 9: YOLO detected 1 people
Processed 10 frames... FPS: 2.63
Frame 10: YOLO detected 1 people
Frame 11: YOLO detected 1 people
Processed 10 frames... FPS: 2.63
Frame 10: YOLO detected 1 people
Frame 11: YOLO detected 1 people
Frame 12: YOLO detected 1 people
Frame 13: YOLO detected 1 people
F

In [3]:
import cv2
from ultralytics import YOLO
from simple_lama_inpainting import SimpleLama
import numpy as np
import subprocess
import os
import time
import socket

# Load YOLO model
yolo_model = YOLO('yolov8n-seg.pt')

# LaMa model (uncomment if available)
lama_model = SimpleLama()

# Dummy inpainting function
def dummy_inpainting(frame, mask):
    # Simple dummy: just return the original frame
    return frame

# Capture a single frame from the raw TCP stream
def capture_frame_from_socket():
    """Capture a single frame from the raw TCP stream on port 5000"""
    s = socket.socket()
    try:
        s.connect(("localhost", 5000))
    except ConnectionRefusedError:
        return None
    W, H = 1280, 720
    BYTES = W * H * 3
    buf = bytearray(BYTES)
    mv = memoryview(buf)
    got = 0
    while got < BYTES:
        n = s.recv_into(mv[got:], BYTES - got)
        if n == 0:
            s.close()
            return None
        got += n
    frame = np.frombuffer(buf, np.uint8).reshape((H, W, 3))
    s.close()
    return frame

# Camera properties (corrected for 720p)
frame_width = 1280
frame_height = 720
fps = 30.0

# Initialize video writers
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out_original = cv2.VideoWriter('./Original_Webcam.avi', fourcc, fps, (frame_width, frame_height))
out_inpainted = cv2.VideoWriter('./SegYolov8_InptSimLama.avi', fourcc, fps, (frame_width, frame_height))

print("Starting video processing with socket capture from raw server stream...")

frame_count = 0
max_frames = 30  # Capture 30 frames

while frame_count < max_frames:
    # Capture frame from socket
    frame = capture_frame_from_socket()
    
    if frame is None:
        print(f"Failed to capture frame {frame_count} from socket")
        break
    
    # Write original frame
    out_original.write(frame)

    # YOLO inference for people (class 0)
    results = yolo_model(frame, classes=[0], verbose=False)

    print(f"Frame {frame_count}: YOLO detected {len(results[0].masks) if results[0].masks is not None else 0} people")

    # For testing LaMa, create a synthetic mask if no people detected
    if results[0].masks is not None and len(results[0].masks) > 0:
        mask = results[0].masks.data[0].cpu().numpy()
        mask = (mask * 255).astype(np.uint8)
        mask = cv2.resize(mask, (frame.shape[1], frame.shape[0]), interpolation=cv2.INTER_NEAREST)
        print(f"Using real mask with shape: {mask.shape}")
    else:
        # Create synthetic circular mask for testing LaMa
        mask = np.zeros((frame.shape[0], frame.shape[1]), dtype=np.uint8)
        center = (frame.shape[1] // 2, frame.shape[0] // 2)
        radius = min(frame.shape[0], frame.shape[1]) // 4
        cv2.circle(mask, center, radius, 255, -1)
        print(f"Using synthetic mask with shape: {mask.shape}")

    # Inpaint using LaMa
    inpainted_result = lama_model(frame, mask)
    print(f"LaMa result type: {type(inpainted_result)}, shape: {getattr(inpainted_result, 'shape', 'no shape')}")
    # Convert PIL Image to numpy array for OpenCV
    if isinstance(inpainted_result, np.ndarray):
        inpainted_frame = inpainted_result
    else:
        # Convert PIL Image to numpy array (BGR format for OpenCV)
        inpainted_frame = np.array(inpainted_result)
        # Convert RGB to BGR if needed
        if len(inpainted_frame.shape) == 3 and inpainted_frame.shape[2] == 3:
            inpainted_frame = cv2.cvtColor(inpainted_frame, cv2.COLOR_RGB2BGR)

    # Write inpainted frame
    out_inpainted.write(inpainted_frame)

    frame_count += 1
    if frame_count % 10 == 0:  # Print progress every 10 frames
        print(f"Processed {frame_count} frames...")

# Release resources
out_original.release()
out_inpainted.release()

print(f"Processing complete! Processed {frame_count} frames from raw server stream.")
print("Videos saved: ./Original_Webcam.avi and ./SegYolov8_InptSimLama.avi")

Starting video processing with socket capture from raw server stream...
Failed to capture frame 0 from socket
Processing complete! Processed 0 frames from raw server stream.
Videos saved: ./Original_Webcam.avi and ./SegYolov8_InptSimLama.avi


In [4]:
# Create side-by-side comparison video
cap_original = cv2.VideoCapture('./Original_Webcam.avi')
cap_inpainted = cv2.VideoCapture('./SegYolov8_InptSimLama.avi')

if not cap_original.isOpened():
    print("Error: Could not open original video file ./Original_Webcam.avi")
    exit()

if not cap_inpainted.isOpened():
    print("Error: Could not open inpainted video file ./SegYolov8_InptSimLama.avi")
    exit()

print("Both video files opened successfully.")
# Get video properties
frame_width = int(cap_original.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap_original.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap_original.get(cv2.CAP_PROP_FPS))

# Create output directory
import os
output_dir = './output'
os.makedirs(output_dir, exist_ok=True)

# Find the next incrementing number
existing_files = [f for f in os.listdir(output_dir) if f.startswith('side_by_side_output_') and f.endswith('.avi')]
numbers = []
for f in existing_files:
    parts = f.split('_')
    if len(parts) >= 4 and parts[-1].endswith('.avi'):
        num_str = parts[-1].replace('.avi', '')
        if num_str.isdigit():
            numbers.append(int(num_str))
next_num = max(numbers) + 1 if numbers else 1

# Define output video path and initialize VideoWriter
output_comparison_path = os.path.join(output_dir, f'side_by_side_output_{next_num}.avi')
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out_comparison = cv2.VideoWriter(output_comparison_path, fourcc, fps, (frame_width * 2, frame_height))

print("Creating side-by-side video...")

# Loop through frames and combine
while cap_original.isOpened() and cap_inpainted.isOpened():
    ret_original, frame_original = cap_original.read()
    ret_inpainted, frame_inpainted = cap_inpainted.read()

    if not ret_original or not ret_inpainted:
        break

    # Concatenate frames horizontally
    combined_frame = np.concatenate((frame_original, frame_inpainted), axis=1)

    # Write the combined frame to the output video
    out_comparison.write(combined_frame)

# Release resources
cap_original.release()
cap_inpainted.release()
out_comparison.release()

print(f"Side-by-side video saved to {output_comparison_path}")

Both video files opened successfully.
Creating side-by-side video...
Side-by-side video saved to ./output/side_by_side_output_4.avi
